# 단일 GPU, 또는 TPU 디바이스를 활용하여 Tunib의 KMWP 데이터셋을 skt-kogpt2-trinity 모델로 학습하는 Base-Code 입니다.  
간단하게 주석을 적었으나, 곳곳에 넣은 링크에서 내용을 자세히 보는 것을 추천드립니다  
처음 접하는 Huggingface가 당황스럽겠지만, 노드에서 배우던 그 BERT나 Transformer 모델을 단 몇줄의  
코드로 사용할 수 있게 해주는 강력한 라이브러리 입니다  

# 프레임워크 및 라이브러리 임포트, TPU 초기화

In [1]:
import tensorflow as tf
from tensorflow.keras.mixed_precision import experimental as mixed_precision
from transformers import AutoTokenizer
from transformers import DataCollatorForLanguageModeling
from transformers import TFGPT2LMHeadModel, AutoConfig

import pandas as pd
from sklearn.model_selection import StratifiedKFold

from tqdm import tqdm
import os

tqdm.pandas() # tqdm 을 pandas 의 DataFrame 에 사용하기 위한 기본 설정

2022-06-03 07:40:09.736546: I tensorflow/core/tpu/tpu_api_dlsym_initializer.cc:116] Libtpu path is: libtpu.so


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2022-06-03 07:40:11.772358: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-03 07:40:15.479637: I tensorflow/compiler/xla/service/service.cc:171] XLA service 0x7a0daf0 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
2022-06-03 07:40:15.479667: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (0): TPU, 2a886c8
2022-06-03 07:40:15.479674: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (1): TPU, 2a886c8
2022-06-03 07:40:15.479679: I tensorflow/compiler/xla/service/service.cc:179]   StreamExecutor device (2): TPU, 2a886c8
2022-06-03 07:40:15.479683: I tensorflow/compiler/xla/service/service.cc:179]   

INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

# 데이터 및 토크나이저 불러오기

In [2]:
# data load
data_path = '/home/dlfrnaos21/data'
train_path = f'{data_path}/new_train.csv'
test_path = f'{data_path}/test.csv'
sample_path = f'{data_path}/sample_answersheet.json'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
sample = pd.read_json(sample_path)

# 기초 전처리 (tokenizer로 tokenize 후 decode하여 unk 토큰이 나오는 단어 변환)
train.problem = train.problem.str.replace("캤","캣")
train.problem = train.problem.str.replace("쨰","째")
train.problem = train.problem.str.replace("츌","출")
train.problem = train.problem.str.replace("찗","짧")
train.problem = train.problem.str.replace("샙","셉")
train.problem = train.problem.str.replace("땃","땄")
train.code = train.code.str.replace("\n","&")

# Base tokenizer 
MODEL_PATH = 'skt/ko-gpt-trinity-1.2B-v0.5'
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH,add_prefix_space=True,
                                          bos_token="<s>",
                                                   eos_token="</s>",
                                                   unk_token="<unk>",
                                                   pad_token="<pad>",
                                                   mask_token="<mask>",)
# Code 문장 tokenize를 위한 code-t5 tokenizer
code_tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")

# 데이터 전처리

In [3]:
train['p_tokenized'] = train.problem.apply(lambda x: tokenizer.encode(x)) # problem to input_ids
train['c_splited'] = train.code.apply(lambda x: code_tokenizer.tokenize("<s> "+x+" </s>")) # code to splited tokens ex:) ['a', '=', '1',...]
train.c_splited = train.c_splited.apply(lambda x: " ".join(x)) # list[str] to string
train.c_joined = train.c_splited.str.replace("Ġ","▁") # replace Ġ to ▁
train.splited_c = train.c_joined.str.split(" ") # split to list[str]
train['c_tokenized'] = train.splited_c.apply(lambda x: tokenizer(x, is_split_into_words=True)['input_ids']) # list[str] to list[int]

/tmp/ipykernel_8806/1527291977.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  train.c_joined = train.c_splited.str.replace("Ġ","▁")
/tmp/ipykernel_8806/1527291977.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  train.splited_c = train.c_joined.str.split(" ")


In [4]:
full_tokenized = train.p_tokenized + train.c_tokenized # add whole tokens
length = full_tokenized.apply(len) # length check
length.describe()

count    2820.000000
mean       74.078723
std        29.936687
min        29.000000
25%        53.750000
50%        65.500000
75%        85.000000
max       234.000000
dtype: float64

In [5]:
# MAX_LEN을 줄이고 BATCH_SIZE를 높이거나, MAX_LEN을 높이고 BATCH_SIZE를 줄이거나 선택
MAX_LEN = 128 

In [6]:
# MAX_LEN 이하의 문장만 데이터로 사용합니다
cut_index = length[length <= MAX_LEN]
cut_text_list = full_tokenized[length <= MAX_LEN].to_list()

[DataCollatorForLanguageModeling](https://huggingface.co/docs/transformers/v4.19.2/en/main_classes/data_collator#transformers.DataCollatorForLanguageModeling)  
language modeling에 사용하는 기능으로 전체 길이에 맞게 input_ids에 padding하며, label을 함께 반환합니다  
GPT는 Causal language modeling을 적용하므로 masked language modeling인 mlm기능은 끄고, input_ids와 동일하나, 패딩만 -100으로 된 label을 반환합니다  
자세한 내용은 huggingface의 [코스](https://huggingface.co/course/en/chapter7/6?fw=tf)를 참고하세요

In [7]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors='tf') # get data_collator object

In [8]:
# return type {"input_ids":[input_ids],"labels":[labels]}
collated_data = data_collator([cut_text_list[i] for i in range(len(cut_text_list))])

In [9]:
# 어텐션 마스크를 직접 만듭니다. input_ids를 기준으로 3(padding token)이 아닌곳은 1을 넣습니다
attention_mask = []
for i in collated_data['input_ids']:
    attention_mask.append([1 if token != 3 else 0 for token in i ])
collated_data['attention_mask'] = tf.convert_to_tensor(attention_mask,dtype=tf.int32)

In [10]:
# Data type 지정을 위한 tf.cast
padded_cut_text_list = tf.cast(collated_data['input_ids'],dtype=tf.int32)
attention_mask = tf.cast(collated_data['attention_mask'],dtype=tf.int32)
labels = tf.cast(collated_data['labels'],dtype=tf.int32)

In [11]:
# 클래스의 분포에 맞게 train과 valid index를 1회만 추출
train_idx, valid_idx = next(iter(StratifiedKFold(n_splits=10, shuffle=True, random_state=5959).split(train.problem[cut_index], train['class'][cut_index])))
train_idx.shape, valid_idx.shape

((2340,), (261,))

In [12]:
# data에서 각 train, valid 에 맞게 데이터를 인덱싱 합니다
train_set = dict(
    input_ids=padded_cut_text_list.numpy()[train_idx],
    attention_mask = attention_mask.numpy()[train_idx],
    labels=labels.numpy()[train_idx],
)
valid_set = dict(
    input_ids=padded_cut_text_list.numpy()[valid_idx],
    attention_mask=attention_mask.numpy()[valid_idx],
    labels=labels.numpy()[valid_idx]
)

# 모델 준비

In [13]:
config = AutoConfig.from_pretrained(MODEL_PATH, n_ctx=MAX_LEN, 
                                                   bos_token_id=tokenizer.bos_token_id, 
                                                   eos_token_id=tokenizer.eos_token_id,
                                                   pad_token_id=tokenizer.pad_token_id,
                                                    )
model = TFGPT2LMHeadModel.from_pretrained(MODEL_PATH, config=config, from_pt=True)
optimizer = tf.keras.optimizers.SGD(momentum=0.9)
model(model.dummy_inputs)
# huggingface 모델의 특징으로, 모델 안에 loss function이 내장되어 있습니다 따라서 따로 설정하지 않아도 loss가 나옵니다
model.compile(optimizer=optimizer,)

# Earlystop 조건으로 자유롭게 바꿀 수 있습니다
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
)

# float32, float16 자료구조를 mix하여 연산하는 것으로 효율적인 연산을 하나, 결과값에 영향이 있을 수 있습니다
# Tensorflow Document 참고 https://www.tensorflow.org/guide/mixed_precision?hl=ko
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

2022-06-03 07:41:27.174125: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:237] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2022-06-03 07:41:27.196530: I tensorflow/compiler/jit/xla_compilation_cache.cc:399] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2022-06-03 07:41:27.300416: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-06-03 07:41:29.985829: W tensorflow/compiler/tf2xla/kernels/random_ops.cc:167] Warning: Using tf.random.truncated_normal with XLA compilation will ignore seeds; consider using tf.random.stateless_truncated_normal instead if reproducible behavior is desired. TruncatedNormal
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.23.attn.masked_bias', 'transformer.h.17.attn.masked_bias', '

# Dataset Pipeline

In [ ]:
# 데이터셋의 입력 파이프라인을 최적화 하는 과정
# 왜 해야하는지 궁금하다면 https://www.tensorflow.org/guide/data_performance?hl=ko
AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 8

def prefetch_ds(ds):
    return ds.repeat().shuffle(2300).batch(BATCH_SIZE,num_parallel_calls=AUTO).prefetch(AUTO)

tf_train_ds = tf.data.Dataset.from_tensor_slices(train_set)
tf_train_ds = prefetch_ds(tf_train_ds)
tf_val_ds = tf.data.Dataset.from_tensor_slices(valid_set)
tf_val_ds = prefetch_ds(tf_val_ds)

train_step = len(train_set['input_ids']) // BATCH_SIZE
val_step = len(valid_set['input_ids']) // BATCH_SIZE
callbacks = [es]

# 모델 학습

In [15]:
history = model.fit(
    dist_train_ds,
    epochs=50,
    callbacks=callbacks,
    validation_data=dist_val_ds,
    steps_per_epoch=train_step,
    validation_steps=val_step,
)



Epoch 1/50


2022-06-03 07:43:03.494078: I tensorflow/core/tpu/graph_rewrite/encapsulate_tpu_computations_pass.cc:263] Subgraph fingerprint:16662598172746951624
2022-06-03 07:43:06.211046: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:828] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2022-06-03 07:43:08.293929: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:828] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2022-06-03 07:43:11.392951: I tensorflow/core/tpu/kernels/tpu_compilation_cache_interface.cc:435] TPU host compilation cache miss: cache_key(1509945416255256015), session_name()
2022-06-03 07:44:41.748193: I tensorflow/core/tpu/kernels/tpu_compile_op_common.cc:180] Compilation of 1509945416255256015 with session name  took 1m30.355153419s and succeeded
2022-06-03 07:44:41.881974: I tensorflow/core/tpu/kernels/tpu_compilation_cache_interface.cc:468] TPU host compilation cache:

18/18 [==============================] - ETA: 0s - loss: 2.0066

2022-06-03 07:44:58.214776: I tensorflow/core/tpu/graph_rewrite/encapsulate_tpu_computations_pass.cc:263] Subgraph fingerprint:11236983988290291293
2022-06-03 07:44:58.818413: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:828] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2022-06-03 07:44:59.380756: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:828] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2022-06-03 07:45:00.176891: I tensorflow/core/tpu/kernels/tpu_compilation_cache_interface.cc:435] TPU host compilation cache miss: cache_key(10690799761149190951), session_name()
2022-06-03 07:45:18.444485: I tensorflow/core/tpu/kernels/tpu_compile_op_common.cc:180] Compilation of 10690799761149190951 with session name  took 18.267515397s and succeeded
2022-06-03 07:45:18.482488: I tensorflow/core/tpu/kernels/tpu_compilation_cache_interface.cc:468] TPU host compilation cache:

18/18 [==============================] - 182s 2s/step - loss: 2.0066 - val_loss: 1.4131
Epoch 2/50
18/18 [==============================] - 12s 682ms/step - loss: 1.3196 - val_loss: 1.1490
Epoch 3/50
18/18 [==============================] - 12s 680ms/step - loss: 1.1385 - val_loss: 1.1267
Epoch 4/50
18/18 [==============================] - 12s 681ms/step - loss: 1.0351 - val_loss: 1.0382
Epoch 5/50
18/18 [==============================] - 12s 683ms/step - loss: 0.9671 - val_loss: 0.9841
Epoch 6/50
18/18 [==============================] - 12s 681ms/step - loss: 0.9119 - val_loss: 0.9966
Epoch 7/50
18/18 [==============================] - 12s 680ms/step - loss: 0.8494 - val_loss: 0.9767
Epoch 8/50
18/18 [==============================] - 12s 683ms/step - loss: 0.8084 - val_loss: 0.9709
Epoch 9/50
18/18 [==============================] - 12s 678ms/step - loss: 0.7564 - val_loss: 0.9704
Epoch 10/50
18/18 [==============================] - 12s 679ms/step - loss: 0.7277 - val_loss: 0.9093
Ep

# Test set 예측 준비

In [21]:
test.problem = test.problem.str.replace("캤","캣")
test.problem = test.problem.str.replace("쨰","째")
test.problem = test.problem.str.replace("츌","출")
test.problem = test.problem.str.replace("찗","짧")
test.problem = test.problem.str.replace("샙","셉")
test.problem = test.problem.str.replace("땃","땄")

In [22]:
input_ids = tokenizer.batch_encode_plus(test.problem.to_list(), padding=True, return_tensors='tf')

In [23]:
AUTO = tf.data.AUTOTUNE
tf_test_ds = tf.data.Dataset.from_tensor_slices(input_ids['input_ids'])
tf_test_ds = tf_test_ds.batch(BATCH_SIZE, num_parallel_calls=AUTO).prefetch(AUTO)

In [24]:
next(iter(tf_test_ds))

<tf.Tensor: shape=(16, 79), dtype=int32, numpy=
array([[30708, 42034, 30206, ...,     3,     3,     3],
       [33104, 19044, 36686, ...,     3,     3,     3],
       [34424, 18792, 30239, ...,     3,     3,     3],
       ...,
       [34657, 30991, 32410, ...,     3,     3,     3],
       [31787, 42501,   565, ...,     3,     3,     3],
       [30473, 29994, 33781, ...,     3,     3,     3]], dtype=int32)>

In [26]:
decoded = []
for i in tf_test_ds:
      output = model.generate(i, max_length=128)
      decoded.append(output)

# 예측 결과 확인

In [27]:
tokenizer.batch_decode(output,skip_special_tokens=True)

['윗접시저울의 왼쪽 접시에 무게가 671g 인 소설책 한 권과 450g 인 과학책 한 권을 올려놓고 오른쪽 접시에 38g 짜리 분동 18개를 올려놓았더니 윗접시저울이 왼 쪽으로 기울었습니다. 윗접시저울이 수평이 되려면 1g 짜리 분동이 몇 개 필요한지 구해 보세요. a = 67 1 & b = 4 50 & c = 38 & d = 18 & y = ( a + b ) // c & print ( y ) ',
 '한 자리 수 A, B가 두 자리 수끼리의 덧셈 식 5A + B2 = 78 를 만족할 때, 두 수 A, B의 합을 구하세요.                        ',
 '한 시간에 오토바이를 15대씩 만드는 공장이 있습니다. 이 공장에서 21시간 동안 만들 수 있는 오토바이는 모두 몇 대가 될까요? 5년간 만들 수 있는 오토바이는 모두 몇 대일까요? ',
 '어떤 두 수의 최대공약수가 64일 때 두 수의 공약수는 모두 몇 개일까요? 앞으로도 ',
 '약분하면 3/4가 되는 분수 중에서 분모와 분자의 차가 12일 때, 분모와 분자의 합을 구해 보세요.  앞으로도 분모와 분자의 차는 12로 일정합니다.  과 같이 분모와 분자의 차를 구하시오. a = 3 / 4 & b = 12 & y = a * b & print (" { :. 2 f }',
 '할머니께서 쪽파를 35개 사 오셨습니다. 이 쪽파을 일주일 동안 매일 같은 수만큼 먹으려고 합니다. 하루에 몇 개씩 먹어야 합니까? 5년간 매일 쪽파를 먹으려면 일주일 동안 쪽파를 몇 개씩 먹어야 합니까? a = 35 & b = 7 & y = a // b & print ( y ) ',
 '어떤 수에서 6를뺀 후 3을 더해야 할 것을 잘못하여 어떤 수에 5를 더한 후 4을 빼었더니 9이 되었습니다. 바르게 계산하면 얼마일까요? 앞으로도 잘못하여 뺐던 수를 구하시오. a = 6 & b = 3 & c = 5 & d = 4 & e = 9 & y = ( e // d + c ) * a & print ( y',
 '4 장의 수 카드 

huggingface model hub를 활용하면 쉽게 모델을 저장하고 불러올 수 있습니다  
단 사전에 huggingface에 가입해야하며, 가입한 아이디의 API 토큰을 가져와서 login시에 뜨는 빈칸에  
토큰을 넣어줘야 작동합니다  
관련 내용은 [huggingface course 4](https://huggingface.co/course/en/chapter4/1?fw=tf)를 참고해주세요


In [ ]:

from huggingface_hub import notebook_login
notebook_login()
!git config --global credential.helper store

In [35]:
# 적절한 repo name과 organization_name을 설정해주세요
os.environ['TOKENIZERS_PARALLELISM'] = "false"
tokenizer.save_pretrained("repository_name",push_to_hub=True,organization="organization_name")
model.save_pretrained("repository_name",push_to_hub=True,organization="organization_name")

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/madatnlp/not_class_trinity-kormath-128
   d05779d..81f1494  main -> main

remote: Allowed refs: all        
To https://huggingface.co/madatnlp/not_class_trinity-kormath-128
   d05779d..81f1494  main -> main



Upload file tf_model.h5:   0%|          | 32.0k/4.33G [00:00<?, ?B/s]

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/madatnlp/not_class_trinity-kormath-128
   81f1494..03e1df2  main -> main

remote: Allowed refs: all        
To https://huggingface.co/madatnlp/not_class_trinity-kormath-128
   81f1494..03e1df2  main -> main

